In [1]:
import numpy as np
import glob

In [2]:
#give every position a coordinate
coor = []
for i in range(9):
    coor.append(np.array([i%3,i//3]))

In [3]:
euclidean = [np.sqrt(np.sum(np.square(t - coor[4]))) for t in coor] 
furthest_location = euclidean.index(max(euclidean))
furthest_location

0

In [4]:
euclidean_map = []
for i in range(len(coor)):
    euclidean_map.append([np.sqrt(np.sum(np.square(t - coor[i]))) for t in coor]) 
euclidean_map = np.array(euclidean_map)

In [5]:
def getCDFMap(p_location,d_t):
    ## calaulate normalized probability for every position and build CDF map
    ## inputs
    #  p_location: the previous location
    #  d_t: time interval
    v_max = np.random.normal(1.2,0.8)
    sigma = v_max * d_t
    co = coor#[t for t in coor if np.sqrt(np.sum(np.square(t - p_location)))<sigma]
    euclidean = [np.sqrt(np.sum(np.square(t - p_location))) for t in co] 
    furthest_location = euclidean.index(max(euclidean))
    p = [np.exp(-(np.sum(np.square(t-p_location)))/(2*sigma**2)) for t in co]
    p_norm = np.array(p)/sum(p)
    for i in range(1,len(p_norm)):
        p_norm[i] += p_norm[i-1]
    return p_norm

In [60]:
def getRotationMatrix(gra,m):
    Ax = gra[0]
    Ay = gra[1]
    Az = gra[2]
    Ex = m[0]
    Ey = m[1]
    Ez = m[2]
    Hx = Ey*Az - Ez*Ay
    Hy = Ez*Ax - Ex*Az
    Hz = Ex*Ay - Ey*Ax
    normH = np.sqrt(Hx*Hx + Hy*Hy + Hz*Hz)
    if normH < 0.1:
        return False
    invH = 1.0 / normH
    Hx *= invH
    Hy *= invH
    Hz *= invH
    invA = 1.0 / np.sqrt(Ax*Ax + Ay*Ay + Az*Az)
    Ax *= invA
    Ay *= invA
    Az *= invA
    Mx = Ay*Hz - Az*Hy
    My = Az*Hx - Ax*Hz
    Mz = Ax*Hy - Ay*Hx
    R = np.array([Hx,Hy,Hz,Mx,My,Mz,Ax,Ay,Az])
    return R

In [61]:
a = 6378137
e = 8.1819190842622e-2
asq = np.power(a,2)
esq = np.power(e,2)
def ecef2lla(ecef):
    x = ecef[0]
    y = ecef[1]
    z = ecef[2]
    r = ecef[3]
    b = np.sqrt( asq * (1-esq) )
    bsq = np.power(b,2)
    ep = np.sqrt( (asq - bsq)/bsq)
    p = np.sqrt(np.power(x,2) + np.power(y,2) )
    th = np.arctan2(a*z, b*p)
    lon = np.arctan2(y,x)
    lat = np.arctan2( (z + np.power(ep,2)*b*np.power(np.sin(th),3) ), (p - esq*a*np.power(np.cos(th),3)) )
    N = a/( np.sqrt(1-esq*np.power(np.sin(lat),2)) )
    alt = p / np.cos(lat) - N
    lon = lon % (2*np.pi)
    ret = [lat, lon, alt , r]
    return ret

In [149]:
def getDatas(location):
    ## get data from a loaction and preprocess
    # thinkng of adding time stamp to get data
    GPSDataDir = glob.glob("transfer_gps/indoor/{:02d}/*.txt".format(location))
    WifiNMagDatas = np.loadtxt('20201022/indoor/wifi/dataRssi_at_{:d}.txt'.format(location))
    WifiNMagChosen = np.random.randint(len(WifiNMagDatas))
    # if timestamp:
    #   wifitimestamp = ?
    #   gpstimestamp = ?
    #   gps chosen = argmin(gpstimestamp - wifitimestamp)
    #print(location)
    #print(len(GPSDataDir))
    GPSChosen = np.random.randint(len(GPSDataDir))
    wifi = WifiNMagDatas[WifiNMagChosen,:15]                     #wifi preprocess
    m = WifiNMagDatas[WifiNMagChosen,-15:-12]/100                #mag preprocess
    gra = WifiNMagDatas[WifiNMagChosen,-3:]/100
    R = getRotationMatrix(gra,m)
    g_mag = np.dot(R.reshape(3,3),m.reshape(3,1))
    f = np.loadtxt(GPSDataDir[GPSChosen],delimiter=",").copy()   #GPS preprocess
    o = np.argsort(f,axis=(0))[:,3]
    f = np.array([ecef2lla(d) for d in f])
    f = f/np.array([(2*np.pi),(2*np.pi),1e+8,1e+14])
    GPS = f[o]
    GPS.resize((10,4))
    return wifi,g_mag, GPS
    

In [155]:
def GenerateRoute(points):
    WifiData = []
    MagData = []
    GPSData = []
    labels = []
    start = np.random.randint(0,8)
    w,m,g = getDatas(start+1)
    WifiData.append(w)
    MagData.append(m)
    GPSData.append(g)
    labels.append(start+1)
    for i in range(points):
        if i ==0:
            #print(start+1)
            NextPos = np.argmax(getCDFMap(coor[start],1)>np.random.rand())
        else:
            NextPos = np.argmax(getCDFMap(coor[NextPos],1)>np.random.rand())
        w,m,g = getDatas(NextPos+1)
        WifiData.append(w)
        MagData.append(m)
        GPSData.append(g)
        labels.append(NextPos+1)
        #print(NextPos+1)
    return np.array(WifiData),np.array(MagData), np.array(GPSData),np.array(labels)

In [170]:
W,M,G,L = GenerateRoute(10)
L.reshape(-1)

array([7, 5, 1, 8, 8, 2, 5, 5, 3, 3, 6])

In [171]:
L.shape

(11,)

In [160]:
def DataGenerator(data_num,steps):
    ## generate train data
    # inputs:
    # data_num:how many route to generate
    # steps: how many location to move in a route
    datas = []
    for i in range(data_num):
        W,M,G,L = GenerateRoute(steps)
        L.reshape()
        for i in range(len(W)):
            #wifi add noise
            mask = np.random.rand((15))
            mask = (mask>0.1).astype(int)
            noise = np.random.normal(scale = 0.05,size=(15))
            W[i]*mask
            W[i]+noise
        #mag normalized
        M = M.astype('float32')
        M = M.reshape(-1,3)
        M = (M - M.mean(axis=0)) / M.std(axis = 0)
        datas.append([W,M,G,L])
    return datas

In [161]:
datas = DataGenerator(10,20)

In [165]:
datas[3][2].shape

(21, 10, 4)